# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [283]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Generate the mathematical model

In [321]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [322]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Production Planning

**Sets and Indices:**

*  $t \in T = \{1, 2, ..., 12\}$: Set of time periods.
*  $f \in F = \{1, 2\}$: Set of food products.
*  $b \in B = \{1, 2, 3\}$: Set of beverage products.

**Parameters:**

*  $FC_{ft}$: Fixed cost of producing one unit of food product $f$ in time period $t$ (from "fixed_cost_production.csv").
*  $VC_{ft}$: Variable cost of producing one unit of food product $f$ in time period $t$ (from "variable_cost_production.csv").
*  $VC_{bt}$: Variable cost of producing one liter of beverage product $b$ in time period $t$ (from "variable_cost_production.csv").
*  $SC_{ft}$: Variable cost of storing one unit of food product $f$ in time period $t$ (from "variable_cost_storage.csv").
*  $SC_{bt}$: Variable cost of storing one liter of beverage product $b$ in time period $t$ (from "variable_cost_storage.csv").
*  $D_{ft}$: Demand for food product $f$ in time period $t$ (from "demand.csv").
*  $D_{bt}$: Demand for beverage product $b$ in time period $t$ (from "demand.csv").
*  $R_{ft}$: Revenue for selling one unit of food product $f$ in time period $t$ (from "revenue.csv").
*  $R_{bt}$: Revenue for selling one liter of beverage product $b$ in time period $t$ (from "revenue.csv").
*  $CapS_f$: Storage capacity for food product $f$ (580 for food 1, 687 for food 2).
*  $CapS_b$: Storage capacity for beverage product $b$ (599L for beverage 1, 788L for beverage 2, 294L for beverage 3).
*  $CapP_f$: Production capacity for food product $f$ (1080 for food 1, 908 for food 2).
*  $CapP_b$: Production capacity for beverage product $b$ (408L for beverage 1, 1000L for beverage 2, 403L for beverage 3).

**Decision Variables:**

*  $P_{ft}$: Quantity of food product $f$ produced in time period $t$.
*  $P_{bt}$: Quantity of beverage product $b$ produced in time period $t$ (in liters).
*  $S_{ft}$: Quantity of food product $f$ stored at the end of time period $t$.
*  $S_{bt}$: Quantity of beverage product $b$ stored at the end of time period $t$ (in liters).
*  $Sales_{ft}$: Quantity of food product $f$ sold in time period $t$.
*  $Sales_{bt}$: Quantity of beverage product $b$ sold in time period $t$ (in liters).

**Objective Function:**

Maximize total profit over the 12 time periods:

 $$Maximize \sum_{t \in T} (\sum_{f \in F}(R_{ft} \cdot Sales_{ft} - FC_{ft} \cdot P_{ft} - VC_{ft} \cdot P_{ft} - SC_{ft} \cdot S_{ft}) + \sum_{b \in B}(R_{bt} \cdot Sales_{bt} - VC_{bt} \cdot P_{bt} - SC_{bt} \cdot S_{bt}))$$

**Constraints:**

1. **Production Capacity:**
   *  $P_{ft} \le CapP_f, \forall f \in F, \forall t \in T$
   *  $P_{bt} \le CapP_b, \forall b \in B, \forall t \in T$

2. **Storage Capacity:**
   *  $S_{ft} \le CapS_f, \forall f \in F, \forall t \in T$
   *  $S_{bt} \le CapS_b, \forall b \in B, \forall t \in T$

3. **Inventory Balance:** 
   *  $S_{f(t-1)} + P_{ft} - Sales_{ft} = S_{ft}, \forall f \in F, \forall t \in T$
   *  $S_{b(t-1)} + P_{bt} - Sales_{bt} = S_{bt}, \forall b \in B, \forall t \in T$
   *  $S_{f0} = 0, \forall f \in F$ 
   *  $S_{b0} = 0, \forall b \in B$

4. **Demand Satisfaction:**
   *  $Sales_{ft} \le D_{ft}, \forall f \in F, \forall t \in T$
   *  $Sales_{bt} \le D_{bt}, \forall b \in B, \forall t \in T$

5. **Non-negativity:**
   *  $P_{ft} \ge 0, \forall f \in F, \forall t \in T$
   *  $P_{bt} \ge 0, \forall b \in B, \forall t \in T$
   *  $S_{ft} \ge 0, \forall f \in F, \forall t \in T$
   *  $S_{bt} \ge 0, \forall b \in B, \forall t \in T$
   *  $Sales_{ft} \ge 0, \forall f \in F, \forall t \in T$
   *  $Sales_{bt} \ge 0, \forall b \in B, \forall t \in T$ 


## 3. Generate the pyomo code

In [323]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [324]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100,
    (1, 2): 110,
    (1, 3): 105,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 120,
    (2, 2): 115,
    (2, 3): 125,
    # ... Add other data points for food product 2 in other time periods
}
variable_cost_production = {
    (1, 1): 10,
    (1, 2): 12,
    (1, 3): 11,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 13,
    (2, 2): 14,
    (2, 3): 15,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 5,  # Beverage 1
    (1, 2): 6,
    # ...
}
variable_cost_storage = {
    (1, 1): 2,
    (1, 2): 3,
    (1, 3): 2.5,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 2.5,
    (2, 2): 3.5,
    (2, 3): 3,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 1,  # Beverage 1
    (1, 2): 1.5,
    # ...
}
demand = {
    (1, 1): 500,
    (1, 2): 550,
    (1, 3): 600,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 400,
    (2, 2): 450,
    (2, 3): 500,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 200,  # Beverage 1
    (1, 2): 250,
    # ... 
}
revenue = {
    (1, 1): 20,
    (1, 2): 22,
    (1, 3): 21,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 25,
    (2, 2): 26,
    (2, 3): 27,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 10,  # Beverage 1
    (1, 2): 11,
    # ...
}


# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, 13))  # Time periods
model.F = pyo.Set(initialize=range(1, 3))  # Food products
model.B = pyo.Set(initialize=range(1, 4))  # Beverage products

# Parameters
model.FC = pyo.Param(model.F, model.T, initialize=fixed_cost_production)
model.VC_f = pyo.Param(model.F, model.T, initialize=variable_cost_production)
model.VC_b = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.SC_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage)
model.SC_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)

# Storage Capacity
model.CapS_f = pyo.Param(model.F, initialize={1: 580, 2: 687}) 
model.CapS_b = pyo.Param(model.B, initialize={1: 599, 2: 788, 3: 294})

# Production Capacity
model.CapP_f = pyo.Param(model.F, initialize={1: 1080, 2: 908}) 
model.CapP_b = pyo.Param(model.B, initialize={1: 408, 2: 1000, 3: 403})

# Decision Variables
model.P_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeReals) # Food production
model.P_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals) # Beverage production
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeReals) # Food storage
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals) # Beverage storage
model.Sales_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeReals) # Food sales
model.Sales_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals) # Beverage sales

# Objective Function
def total_profit(model):
    return sum(
        (model.R_f[f, t] * model.Sales_f[f, t] - model.FC[f, t] * model.P_f[f, t] -
         model.VC_f[f, t] * model.P_f[f, t] - model.SC_f[f, t] * model.S_f[f, t]) + 
        (model.R_b[b, t] * model.Sales_b[b, t] - model.VC_b[b, t] * model.P_b[b, t] -
         model.SC_b[b, t] * model.S_b[b, t]) 
        for f in model.F for b in model.B for t in model.T
    )

model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# 1. Production Capacity
def production_capacity_food(model, f, t):
    return model.P_f[f, t] <= model.CapP_f[f]
model.production_capacity_food_constraint = pyo.Constraint(model.F, model.T, rule=production_capacity_food)

def production_capacity_beverage(model, b, t):
    return model.P_b[b, t] <= model.CapP_b[b]
model.production_capacity_beverage_constraint = pyo.Constraint(model.B, model.T, rule=production_capacity_beverage)

# 2. Storage Capacity
def storage_capacity_food(model, f, t):
    return model.S_f[f, t] <= model.CapS_f[f]
model.storage_capacity_food_constraint = pyo.Constraint(model.F, model.T, rule=storage_capacity_food)

def storage_capacity_beverage(model, b, t):
    return model.S_b[b, t] <= model.CapS_b[b]
model.storage_capacity_beverage_constraint = pyo.Constraint(model.B, model.T, rule=storage_capacity_beverage)

# 3. Inventory Balance
def inventory_balance_food(model, f, t):
    if t == 1:
        return model.P_f[f, t] - model.Sales_f[f, t] == model.S_f[f, t]
    else:
        return model.S_f[f, t-1] + model.P_f[f, t] - model.Sales_f[f, t] == model.S_f[f, t]
model.inventory_balance_food_constraint = pyo.Constraint(model.F, model.T, rule=inventory_balance_food)

def inventory_balance_beverage(model, b, t):
    if t == 1:
        return model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]
    else:
        return model.S_b[b, t-1] + model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]
model.inventory_balance_beverage_constraint = pyo.Constraint(model.B, model.T, rule=inventory_balance_beverage)

# 4. Demand Satisfaction
def demand_satisfaction_food(model, f, t):
    return model.Sales_f[f, t] <= model.D_f[f, t]
model.demand_satisfaction_food_constraint = pyo.Constraint(model.F, model.T, rule=demand_satisfaction_food)

def demand_satisfaction_beverage(model, b, t):
    return model.Sales_b[b, t] <= model.D_b[b, t]
model.demand_satisfaction_beverage_constraint = pyo.Constraint(model.B, model.T, rule=demand_satisfaction_beverage)

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can use other solvers like 'gurobi', 'cplex' etc.
solver.solve(model)

# Print the results
print("Profit = ", model.profit())

# You can print the values of other variables like this
# print("Food Production (P_f) = ", model.P_f.get_values()) 
# ... etc.
```

**Explanation:**

1.  **Import Pyomo:**  `import pyomo.environ as pyo`
2.  **Sample Data:** The code starts by defining sample data for the parameters. **Remember to replace this sample data with the actual data from your CSV files.**
3.  **Create a Model:** A `ConcreteModel` is created to represent the optimization problem.
4.  **Define Sets:** Sets for time periods (T), food products (F), and beverage products (B) are created using `pyo.Set`.
5.  **Define Parameters:** The parameters from the problem are defined using `pyo.Param` and initialized with the sample data.
6.  **Define Decision Variables:** Decision variables are defined using `pyo.Var` and are restricted to be non-negative using `domain=pyo.NonNegativeReals`.
7.  **Define Objective Function:** The objective function, which is to maximize profit, is defined using `pyo.Objective`.
8.  **Define Constraints:**  Each constraint from the mathematical model is implemented using `pyo.Constraint`.
9.  **Solve the Model:** A solver ('cbc' in this case) is selected using `pyo.SolverFactory` and then used to solve the model.
10. **Print Results:** The optimal profit and values of decision variables can be printed using the `print()` function and accessing the variable values using `.get_values()`.

**To Use Your Data:**

1.  **Install Pyomo:**  You can install Pyomo using `pip install pyomo`.
2.  **Import Data from CSV:** You can use libraries like `pandas` to import data from your CSV files.
3.  **Replace Sample Data:** Replace the sample data in the code with the data you imported from your CSV files.

Remember to replace the sample data with your own data, and you might need to install a solver like CBC, Gurobi, or CPLEX depending on the complexity of your problem.


## 4. Input problem data and try running the generated code

In [332]:
import pyomo.environ as pyo

# # Sample Data (Replace with actual data from CSV files)
# fixed_cost_production = {
#     (1, 1): 100,
#     (1, 2): 110,
#     (1, 3): 105,
#     # ... Add other data points for food product 1 in other time periods
#     (2, 1): 120,
#     (2, 2): 115,
#     (2, 3): 125,
#     # ... Add other data points for food product 2 in other time periods
# }
# variable_cost_production = {
#     (1, 1): 10,
#     (1, 2): 12,
#     (1, 3): 11,
#     # ... Add other data points for food product 1 in other time periods
#     (2, 1): 13,
#     (2, 2): 14,
#     (2, 3): 15,
#     # ... Add other data points for food product 2 in other time periods
#     (1, 1): 5,  # Beverage 1
#     (1, 2): 6,
#     # ...
# }
# variable_cost_storage = {
#     (1, 1): 2,
#     (1, 2): 3,
#     (1, 3): 2.5,
#     # ... Add other data points for food product 1 in other time periods
#     (2, 1): 2.5,
#     (2, 2): 3.5,
#     (2, 3): 3,
#     # ... Add other data points for food product 2 in other time periods
#     (1, 1): 1,  # Beverage 1
#     (1, 2): 1.5,
#     # ...
# }
# demand = {
#     (1, 1): 500,
#     (1, 2): 550,
#     (1, 3): 600,
#     # ... Add other data points for food product 1 in other time periods
#     (2, 1): 400,
#     (2, 2): 450,
#     (2, 3): 500,
#     # ... Add other data points for food product 2 in other time periods
#     (1, 1): 200,  # Beverage 1
#     (1, 2): 250,
#     # ... 
# }
# revenue = {
#     (1, 1): 20,
#     (1, 2): 22,
#     (1, 3): 21,
#     # ... Add other data points for food product 1 in other time periods
#     (2, 1): 25,
#     (2, 2): 26,
#     (2, 3): 27,
#     # ... Add other data points for food product 2 in other time periods
#     (1, 1): 10,  # Beverage 1
#     (1, 2): 11,
#     # ...
# }

#LOADING DATA BY HUMAN
import pandas as pd

# Read the dataframes
fixed_cost_production = pd.read_csv("fixed_cost_production.csv")
variable_cost_production = pd.read_csv("variable_cost_production.csv")
variable_cost_storage = pd.read_csv("variable_cost_storage.csv")
demand = pd.read_csv("demand.csv")
revenue = pd.read_csv("revenue.csv")

# Define the number of food and beverage items
num_foods = 2
num_beverages = len(fixed_cost_production) - num_foods

# Transform into dictionary format
fixed_cost_production_foods = {(i, j): fixed_cost_production.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(fixed_cost_production.columns))}
fixed_cost_production_beverages = {(i, j): fixed_cost_production.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(fixed_cost_production.columns))}
variable_cost_production_foods = {(i, j): variable_cost_production.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_production_beverages = {(i, j): variable_cost_production.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_storage_foods = {(i, j): variable_cost_storage.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(variable_cost_storage.columns))}
variable_cost_storage_beverages = {(i, j): variable_cost_storage.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(variable_cost_storage.columns))}
demand_foods = {(i, j): demand.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(demand.columns))}
demand_beverages = {(i, j): demand.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(demand.columns))}
revenue_foods = {(i, j): revenue.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(revenue.columns))}
revenue_beverages = {(i, j): revenue.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(revenue.columns))}

#END

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, 13))  # Time periods
model.F = pyo.Set(initialize=range(1, 3))  # Food products
model.B = pyo.Set(initialize=range(1, 4))  # Beverage products

# Parameters
model.FC = pyo.Param(model.F, model.T, initialize=fixed_cost_production_foods)
model.VC_f = pyo.Param(model.F, model.T, initialize=variable_cost_production_foods)
model.VC_b = pyo.Param(model.B, model.T, initialize=variable_cost_production_beverages)
model.SC_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage_foods)
model.SC_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage_beverages)
model.D_f = pyo.Param(model.F, model.T, initialize=demand_foods)
model.D_b = pyo.Param(model.B, model.T, initialize=demand_beverages)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue_foods)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue_beverages)

# Storage Capacity
model.CapS_f = pyo.Param(model.F, initialize={1: 580, 2: 687}) 
model.CapS_b = pyo.Param(model.B, initialize={1: 599, 2: 788, 3: 294})

# Production Capacity
model.CapP_f = pyo.Param(model.F, initialize={1: 1080, 2: 908}) 
model.CapP_b = pyo.Param(model.B, initialize={1: 408, 2: 1000, 3: 403})

# Decision Variables
model.P_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeReals) # Food production
model.P_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals) # Beverage production
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeReals) # Food storage
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals) # Beverage storage
model.Sales_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeReals) # Food sales
model.Sales_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals) # Beverage sales

# Objective Function
def total_profit(model):
    return sum(
        (model.R_f[f, t] * model.Sales_f[f, t] - model.FC[f, t] * model.P_f[f, t] -
         model.VC_f[f, t] * model.P_f[f, t] - model.SC_f[f, t] * model.S_f[f, t]) + 
        (model.R_b[b, t] * model.Sales_b[b, t] - model.VC_b[b, t] * model.P_b[b, t] -
         model.SC_b[b, t] * model.S_b[b, t]) 
        for f in model.F for b in model.B for t in model.T
    )

model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# 1. Production Capacity
def production_capacity_food(model, f, t):
    return model.P_f[f, t] <= model.CapP_f[f]
model.production_capacity_food_constraint = pyo.Constraint(model.F, model.T, rule=production_capacity_food)

def production_capacity_beverage(model, b, t):
    return model.P_b[b, t] <= model.CapP_b[b]
model.production_capacity_beverage_constraint = pyo.Constraint(model.B, model.T, rule=production_capacity_beverage)

# 2. Storage Capacity
def storage_capacity_food(model, f, t):
    return model.S_f[f, t] <= model.CapS_f[f]
model.storage_capacity_food_constraint = pyo.Constraint(model.F, model.T, rule=storage_capacity_food)

def storage_capacity_beverage(model, b, t):
    return model.S_b[b, t] <= model.CapS_b[b]
model.storage_capacity_beverage_constraint = pyo.Constraint(model.B, model.T, rule=storage_capacity_beverage)

# 3. Inventory Balance
def inventory_balance_food(model, f, t):
    if t == 1:
        return model.P_f[f, t] - model.Sales_f[f, t] == model.S_f[f, t]
    else:
        return model.S_f[f, t-1] + model.P_f[f, t] - model.Sales_f[f, t] == model.S_f[f, t]
model.inventory_balance_food_constraint = pyo.Constraint(model.F, model.T, rule=inventory_balance_food)

def inventory_balance_beverage(model, b, t):
    if t == 1:
        return model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]
    else:
        return model.S_b[b, t-1] + model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]
model.inventory_balance_beverage_constraint = pyo.Constraint(model.B, model.T, rule=inventory_balance_beverage)

# 4. Demand Satisfaction
def demand_satisfaction_food(model, f, t):
    return model.Sales_f[f, t] <= model.D_f[f, t]
model.demand_satisfaction_food_constraint = pyo.Constraint(model.F, model.T, rule=demand_satisfaction_food)

def demand_satisfaction_beverage(model, b, t):
    return model.Sales_b[b, t] <= model.D_b[b, t]
model.demand_satisfaction_beverage_constraint = pyo.Constraint(model.B, model.T, rule=demand_satisfaction_beverage)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi', 'cplex' etc.
solver.solve(model)

# Print the results
print("Profit = ", model.profit())

# You can print the values of other variables like this
# print("Food Production (P_f) = ", model.P_f.get_values()) 
# ... etc.

Profit =  16317.2966


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [333]:
print(response.text)

## Mathematical Optimization Model for Production Planning

**Sets and Indices:**

*  $t \in T = \{1, 2, ..., 12\}$: Set of time periods.
*  $f \in F = \{1, 2\}$: Set of food products.
*  $b \in B = \{1, 2, 3\}$: Set of beverage products.

**Parameters:**

*  $FC_{ft}$: Fixed cost of producing one unit of food product $f$ in time period $t$ (from "fixed_cost_production.csv").
*  $VC_{ft}$: Variable cost of producing one unit of food product $f$ in time period $t$ (from "variable_cost_production.csv").
*  $VC_{bt}$: Variable cost of producing one liter of beverage product $b$ in time period $t$ (from "variable_cost_production.csv").
*  $SC_{ft}$: Variable cost of storing one unit of food product $f$ in time period $t$ (from "variable_cost_storage.csv").
*  $SC_{bt}$: Variable cost of storing one liter of beverage product $b$ in time period $t$ (from "variable_cost_storage.csv").
*  $D_{ft}$: Demand for food product $f$ in time period $t$ (from "demand.csv").
*  $D_{bt}$: Demand for beve

In [334]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100,
    (1, 2): 110,
    (1, 3): 105,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 120,
    (2, 2): 115,
    (2, 3): 125,
    # ... Add other data points for food product 2 in other time periods
}
variable_cost_production = {
    (1, 1): 10,
    (1, 2): 12,
    (1, 3): 11,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 13,
    (2, 2): 14,
    (2, 3): 15,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 5,  # Beverage 1
    (1, 2): 6,
    # ...
}
variable_cost_storage = {
    (1, 1): 2,
    (1, 2): 3,
    (1, 3): 2.5,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 2.5,
    (2, 2): 3.5,
    (2, 3): 3,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 1,  # Beverage 1
    (